In [1]:
!python -V

Python 3.9.19


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

print(f"sklearn version: {sklearn.__version__}")

import warnings
warnings.filterwarnings("ignore")

sklearn version: 1.4.2


In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/gan-m2/MLOps/Module 2/mlruns/1', creation_time=1716865717048, experiment_id='1', last_update_time=1716865717048, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715211934021

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
# 开启一个新的 MLflow 运行，在 with 代码块内的所有 MLflow 相关操作都会记录到这个运行中
with mlflow.start_run():

    # 标签 developer，值为 cristian
    mlflow.set_tag("developer", "cristian")

    # 记录训练数据和验证数据的路径。
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    # 记录超参数 alpha 的值。这里 alpha 是用于 Lasso 回归模型的正则化强度。
    alpha = 0.2
    mlflow.log_param("alpha", alpha)
    
    # 建模
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    # 使用验证数据 X_val 进行预测，并计算预测结果的均方根误差（RMSE）。然后将 RMSE 作为指标记录下来
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    # 将本地路径为 models/lin_reg.bin 的模型文件记录为一个工件，存储到 models_pickle 目录中。工件可以是模型文件、数据文件等，便于日后下载和使用。

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 单个xgboost模型训练

In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
# param: 在超参数优化过程中，Hyperopt 会从这个搜索空间中采样参数组合，并将其传递给 objective 函数中的 params 参数。
# 因此，params 是从搜索空间中采样得到的一组具体参数，用于配置和训练 XGBoost 模型。
# 也即是下面的 search_space 
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

### Hyper tuning and store logs in MLflow (it should run 2 hours + to complete)

In [24]:
# max_depth: 树的最大深度，取值范围为 4 到 100 之间的整数。
# learning_rate: 学习率，取值范围为 0.001 到 1 之间的对数均匀分布。
# reg_alpha: L1 正则化项，取值范围为 0.00001 到 0.1 之间的对数均匀分布。
# reg_lambda: L2 正则化项，取值范围为 0.000001 到 0.1 之间的对数均匀分布。
# min_child_weight: 最小子节点权重，取值范围为 0.1 到 20 之间的对数均匀分布。
# objective: 目标函数，这里设置为回归任务。
# seed: 随机种子，用于保证结果的可重复性。

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# fmin: 优化函数
# fn: 目标函数，也就是我们在上面定义的 function
# space: 参数空间，也就是我们要搜索的超参数范围，在上面定义的 search_space
# algo: 优化算法，这里使用 TPE 算法，也就是树形 Parzen Estimator
# max_evals: 最大迭代次数
# trials: 用于记录每次迭代的结果
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:7.18551                           
[1]	validation-rmse:6.74890                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:55:28] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:6.68660                           
[3]	validation-rmse:6.67631                           
[4]	validation-rmse:6.66562                           
[5]	validation-rmse:6.66060                           
[6]	validation-rmse:6.65793                           
[7]	validation-rmse:6.65051                           
[8]	validation-rmse:6.63889                           
[9]	validation-rmse:6.63304                           
[10]	validation-rmse:6.62931                          
[11]	validation-rmse:6.62501                          
[12]	validation-rmse:6.61594                          
[13]	validation-rmse:6.60854                          
[14]	validation-rmse:6.60432                          
[15]	validation-rmse:6.60278                          
[16]	validation-rmse:6.59901                          
[17]	validation-rmse:6.59760                          
[18]	validation-rmse:6.59351                          
[19]	validation-rmse:6.58840                          
[20]	valid

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:55:39] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.19682                                                    
[1]	validation-rmse:7.05841                                                    
[2]	validation-rmse:6.74699                                                    
[3]	validation-rmse:6.64427                                                    
[4]	validation-rmse:6.60503                                                    
[5]	validation-rmse:6.59068                                                    
[6]	validation-rmse:6.58134                                                    
[7]	validation-rmse:6.57321                                                    
[8]	validation-rmse:6.56684                                                    
[9]	validation-rmse:6.56086                                                    
[10]	validation-rmse:6.55716                                                   
[11]	validation-rmse:6.55255                                                   
[12]	validation-rmse:6.54896            

...

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:25:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[6]	validation-rmse:7.86570                                                      
[7]	validation-rmse:7.63517                                                      
[8]	validation-rmse:7.45561                                                      
[9]	validation-rmse:7.31476                                                      
[10]	validation-rmse:7.20221                                                     
[11]	validation-rmse:7.11289                                                     
[12]	validation-rmse:7.04302                                                     
[13]	validation-rmse:6.98696                                                     
[14]	validation-rmse:6.94194                                                     
[15]	validation-rmse:6.90601                                                     
[16]	validation-rmse:6.87725                                                     
[17]	validation-rmse:6.85423                                                     
[18]	validation-

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:25:45] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.35745                                                     
[1]	validation-rmse:10.61783                                                     
[2]	validation-rmse:9.98177                                                      
[3]	validation-rmse:9.43774                                                      
[4]	validation-rmse:8.97166                                                      
[5]	validation-rmse:8.57741                                                      
[6]	validation-rmse:8.24308                                                      
[7]	validation-rmse:7.96187                                                      
[8]	validation-rmse:7.72604                                                      
 88%|████████▊ | 44/50 [30:20<04:08, 41.38s/trial, best loss: 6.3050429894846625]


KeyboardInterrupt: 

### fit a model with optimal parameters selected above (with auto_log=True)

[MLflow Tracking](https://mlflow.org/docs/latest/tracking/autolog.html#supported-libraries)

The generic autolog function mlflow.autolog() enables autologging for each supported library you have installed as soon as you import it.

也就是说，只要你import了`mlflow`，然后调用[`mlflow.autolog()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.autolog)，那么你在后面调用的任何支持的库，都会自动记录参数，模型，metrics等等。

The following libraries support autologging:
- Fastai
- Gluon
- Keras
- LightGBM
- PyTorch
- Scikit-learn
- Spark
- Statsmodels
- XGBoost


In [63]:
# 启用XGBoost自动记录
mlflow.xgboost.autolog()

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.058550748798202254,
        'max_depth': 16,
        'min_child_weight': 3.616990741399196,
        'objective': 'reg:linear',
        'reg_alpha': 0.014691513505959162,
        'reg_lambda': 0.15590387336160477,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # 当auto_log=False时，需要手动调用mlflow.log_model
    # with open("models/preprocessor.b", "wb") as f_out:
    #     pickle.dump(dv, f_out)
    # mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.74481


/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:54:09] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:11.31211
[2]	validation-rmse:10.91238
[3]	validation-rmse:10.54417
[4]	validation-rmse:10.20505
[5]	validation-rmse:9.89290
[6]	validation-rmse:9.60570
[7]	validation-rmse:9.34219
[8]	validation-rmse:9.10103
[9]	validation-rmse:8.88014
[10]	validation-rmse:8.67854
[11]	validation-rmse:8.49462
[12]	validation-rmse:8.32646
[13]	validation-rmse:8.17389
[14]	validation-rmse:8.03496
[15]	validation-rmse:7.90781
[16]	validation-rmse:7.79301
[17]	validation-rmse:7.68817
[18]	validation-rmse:7.59198
[19]	validation-rmse:7.50505
[20]	validation-rmse:7.42664
[21]	validation-rmse:7.35457
[22]	validation-rmse:7.28971
[23]	validation-rmse:7.23062
[24]	validation-rmse:7.17669
[25]	validation-rmse:7.12811
[26]	validation-rmse:7.08421
[27]	validation-rmse:7.04428
[28]	validation-rmse:7.00817
[29]	validation-rmse:6.97509
[30]	validation-rmse:6.94465
[31]	validation-rmse:6.91717
[32]	validation-rmse:6.89209
[33]	validation-rmse:6.86916
[34]	validation-rmse:6.84837
[35]	validation-rms

2024/05/28 15:54:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/28 15:54:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure

### fit a model with optimal parameters selected above (with auto_log=False)


In [65]:
# 不启用XGBoost自动记录
mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.058550748798202254,
        'max_depth': 16,
        'min_child_weight': 3.616990741399196,
        'objective': 'reg:linear',
        'reg_alpha': 0.014691513505959162,
        'reg_lambda': 0.15590387336160477,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # 将上面的dv保存到models/preprocessor.b文件中，
    # 因为我们使用了X_train来fit了预处理器 dv = DictVectorizer(), 当我们再次使用模型时，需要使用相同的预处理器
    with open("models/preprocessor.b", "wb") as f_out:
        # 用pickle包保存dv，其中dv是DictVectorizer()的实例，而f_out是一个文件对象
        # 这俩共同保存在了models/preprocessor.b文件中
        pickle.dump(dv, f_out)
    
    # 当auto_log=False时，需要手动添加mlflow.artifact，从而记录模型文件
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.74481
[1]	validation-rmse:11.31211
[2]	validation-rmse:10.91238
[3]	validation-rmse:10.54417
[4]	validation-rmse:10.20505
[5]	validation-rmse:9.89290
[6]	validation-rmse:9.60570
[7]	validation-rmse:9.34219
[8]	validation-rmse:9.10103
[9]	validation-rmse:8.88014
[10]	validation-rmse:8.67854
[11]	validation-rmse:8.49462
[12]	validation-rmse:8.32646
[13]	validation-rmse:8.17389
[14]	validation-rmse:8.03496
[15]	validation-rmse:7.90781
[16]	validation-rmse:7.79301
[17]	validation-rmse:7.68817
[18]	validation-rmse:7.59198
[19]	validation-rmse:7.50505
[20]	validation-rmse:7.42664
[21]	validation-rmse:7.35457
[22]	validation-rmse:7.28971
[23]	validation-rmse:7.23062
[24]	validation-rmse:7.17669
[25]	validation-rmse:7.12811
[26]	validation-rmse:7.08421
[27]	validation-rmse:7.04428
[28]	validation-rmse:7.00817
[29]	validation-rmse:6.97509
[30]	validation-rmse:6.94465
[31]	validation-rmse:6.91717
[32]	validation-rmse:6.89209
[33]	validation-rmse:6.86916
[34]	validation-rms

In [ ]:
import pickle

# 加载预处理器
with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

# 使用加载的预处理器进行数据转换
# transformed_data = dv.transform(new_data)

### 不用autolog的方法（原来的手动方法）

In [26]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.058550748798202254,
        'max_depth': 16,
        'min_child_weight': 3.616990741399196,
        'objective': 'reg:linear',
        'reg_alpha': 0.014691513505959162,
        'reg_lambda': 0.15590387336160477,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    # 不需要再次手动调用mlflow.xgboost.log_model
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.74481


/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:45:15] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:11.31211
[2]	validation-rmse:10.91238
[3]	validation-rmse:10.54417
[4]	validation-rmse:10.20505
[5]	validation-rmse:9.89290
[6]	validation-rmse:9.60570
[7]	validation-rmse:9.34219
[8]	validation-rmse:9.10103
[9]	validation-rmse:8.88014
[10]	validation-rmse:8.67854
[11]	validation-rmse:8.49462
[12]	validation-rmse:8.32646
[13]	validation-rmse:8.17389
[14]	validation-rmse:8.03496
[15]	validation-rmse:7.90781
[16]	validation-rmse:7.79301
[17]	validation-rmse:7.68817
[18]	validation-rmse:7.59198
[19]	validation-rmse:7.50505
[20]	validation-rmse:7.42664
[21]	validation-rmse:7.35457
[22]	validation-rmse:7.28971
[23]	validation-rmse:7.23062
[24]	validation-rmse:7.17669
[25]	validation-rmse:7.12811
[26]	validation-rmse:7.08421
[27]	validation-rmse:7.04428
[28]	validation-rmse:7.00817
[29]	validation-rmse:6.97509
[30]	validation-rmse:6.94465
[31]	validation-rmse:6.91717
[32]	validation-rmse:6.89209
[33]	validation-rmse:6.86916
[34]	validation-rmse:6.84837
[35]	validation-rms

2024/05/27 23:45:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:45:56] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/27 23:45:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure

### Predict on a # Load model as a PyFuncModel （方法1）

In [56]:
# Load model as a PyFuncModel.
logged_model = 'runs:/c51562cf17cb433fbb1deb05b0709d49/model'

loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:26:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: c51562cf17cb433fbb1deb05b0709d49

In [58]:
X_val_dict = df_val[categorical + numerical].to_dict(orient='records')
# preprocessed DataFrame with DictionaryVectorizer
X_val = dv.transform(X_val_dict)

In [60]:
# loaded_model.predict(valid) # 这个会报错，因为loaded_model是一个PyFuncModel，不是XGBoost模型
# error内容显示 TypeError: Not supported type for data.<class 'xgboost.core.DMatrix'>
loaded_model.predict(X_val)

array([15.085921,  7.206091, 16.032732, ..., 15.449408, 27.112658,
        8.331218], dtype=float32)

### Load model as XGBoost Model (方法2)

In [61]:
# Load model as XGBmodel
xgb_model = mlflow.xgboost.load_model(logged_model)
xgb_model

/Users/gan-m2/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:30:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [62]:
# convert X_val into DMatrix type
valid = xgb.DMatrix(X_val, label=y_val)

# 当我们使用XGBoost模型时，我们需要使用刚才转换的DMatrix类型的数据，而不是原始的特征矩阵。
xgb_model.predict(valid)

array([15.085921,  7.206091, 16.032732, ..., 15.449408, 27.112658,
        8.331218], dtype=float32)

### 使用不同的模型

In [80]:
# x爱你在的MLflow仅支持 scikit-learn <= 1.4.2
# downgrade scikit-learn to 1.4.2
# 先卸载当前版本
!pip uninstall -y scikit-learn

Found existing installation: scikit-learn 1.4.2
Uninstalling scikit-learn-1.4.2:
  Successfully uninstalled scikit-learn-1.4.2


In [ ]:
# !pip install scikit-learn==1.4.2

In [11]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor

from sklearn.svm import LinearSVR

In [17]:
# mlflow.sklearn.autolog()

# 因为tmd太慢了所以我没跑这个
# RandomForestRegressor, ExtraTreesRegressor 超级慢
# LinearSVR 我觉得这个也很慢

# for model_class in (GradientBoostingRegressor):

#     with mlflow.start_run():

#         mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#         mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
#         mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#         mlmodel = model_class()
#         mlmodel.fit(X_train, y_train)

#         y_pred = mlmodel.predict(X_val)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
# import lasso and ElasticNet
from sklearn.linear_model import Lasso, ElasticNet

In [12]:
# 这个一定要设置，不然MLflow不会记录模型
# 这里我们重新安装了一遍sklearn==1.4.2，因为MLflow仅支持 scikit-learn <= 1.4.2 （2024-05-28）
mlflow.sklearn.autolog()

In [16]:
# 这三个跑得快，原本的跑的太jbd慢了
for model_class in (GradientBoostingRegressor, ElasticNet, Lasso):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)


2024/05/28 17:19:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 17:19:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 17:19:46 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
